w_03 | Detección de APIs privadas
====

Supongamos que queremos bajar el tipo de cambio diario del Banco Central porque nuestro negocio tiene operaciones en dólares. Si vamos al [BCU](https://www.bcu.gub.uy/Estadisticas-e-Indicadores/Paginas/Cotizaciones.aspx) pasan dos cosas:

1. Hay un captcha.
2. La interfaz no nos deja descargar más que un año de datos para una moneda, o más de un mes para un conjunto de monedas.

Esto no es para nada cómodo si pretendemos hacer este proceso de manera frecuente o si necesitamos construir una serie de datos larga.

Probablemente haya alguna API corriendo por detrás que deberíamos poder encontrar y explotar. Así que abrimos el panel de desarrollador, seleccionamos un rango de fechas y algunas monedas y ejecutamos la consulta de Excel o la de Mostrar.

![](https://drive.google.com/uc?export=view&id=14Y4q0_-7offYl8ljtulR_XsGmoJzbyhW)

Eso inmediatamente va a generar cambios en el panel de desarrollador, específicamente en la pestaña Network. Lo que sigue es un poco artesanal, pero consiste en encontrar la consulta que específicamente refiere a la generación de los datos.

En este caso corrí específicamente la consulta de Mostrar. Al abrirla tenemos que registrar el protocolo, headers y cualquier payload que se haya enviado.

![](https://drive.google.com/uc?export=view&id=1wIoM8_eu1k2zIs4E3ki4x43lpsPIXsKA)

![](https://drive.google.com/uc?export=view&id=1kVCikF9lEWjswg9Gqw9EB2aiqTFE5E33)

Así que ya sabemos el protocolo, los headers y los datos que se envían al servidor. Ahora hay que transformar eso en un request y ver qué devuelve el servidor.

In [ ]:
import requests
import pandas as pd
from pprint import pprint

url = "https://www.bcu.gub.uy/_layouts/15/BCU.Cotizaciones/handler/CotizacionesHandler.ashx?op=getcotizaciones"
payload = {"KeyValuePairs":
            {"Monedas":[{"Val":"501","Text":"PESO ARG.BILLETE"}, {"Val":"1001","Text":"REAL BILLETE"}, {"Val":"2230","Text":"DLS.PROMED.FONDO"}],
             "FechaDesde":"01/04/2021",
             "FechaHasta":"20/04/2021",
             "Grupo":"2"}}
r = requests.post(url, json=payload)
r.text

Algunos comentarios:

1. No pasamos directamente el diccionario al argumento ``data`` de ``get()``, sino que lo hicimos a ``json``. Podríamos haber codificado el diccionario en JSON usando la librería standard de Python, pero esto nos ahorra un paso. No todas las payloads tienen que enviarse en JSON, eso depende del servidor.
2. La respuesta tiene toda la pinta de ser un JSON, así que podemos decodificarlo directamente con ``requests`` para que nos quede un diccionario.

In [ ]:
pprint(r.json())

Lo cual parece estar bien, pero no es útil desde un punto de vista de análisis de datos. Necesitamos meterlo en una extructura tabular, para lo cual directamente vamos a tirarle el diccionario al constructor de ``pd.DataFrame``.

Notar que ingresamos a la key "cotizacionesoutlist" y luego a "Cotizaciones" para que nos devuelva registros individuales.

In [ ]:
df = pd.DataFrame(r.json()["cotizacionesoutlist"]["Cotizaciones"])
df.head(10)

Eso es todo, descubrimos la API del Banco Central para consultar monedas. Dado que sabemos la estructura que toma el payload, podemos ir variando fechas para construir la base de datos mes a mes, lo que esquiva la restricción que la web impone.